<a href="https://colab.research.google.com/github/nonohuang0819/kaggle/blob/main/task2/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np 
import pandas as pd 
import os
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
import category_encoders as ce
from sklearn.utils import shuffle
#from google.colab import drive
#drive.mount("/content/drive")

In [49]:
#filepath = "/content/drive/MyDrive/113-1機器學習概論/KaggleContest/kaggle/task2/introml_2024_task2_train.csv"
filepath = r"C:\Users\nonohuang\OneDrive\桌面\kaggle\kaggle\task2\introml_2024_task2_train.csv"
df = pd.read_csv(filepath)
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,class
0,f00,f11,f20,f30,f40,f50,f61,f71,f81,0.3839,1.5223,0.5516,0.0579,2.2677,1.1361,0.9204,1.8718,0.7433,C0
1,f00,f10,f20,f30,f40,f50,f61,f70,f81,1.4089,1.7259,0.9097,1.5688,0.9065,1.2314,0.9521,1.5515,0.7560,C0
2,f01,f10,f20,f30,f41,f50,f60,f71,f80,0.2752,0.2936,0.5023,1.6234,1.7168,1.3118,0.3520,2.5584,2.0897,C0
3,f01,f11,f20,f31,f41,f50,f60,f70,f80,0.1777,0.1441,2.1431,1.3822,2.1976,0.7040,0.1160,0.1408,2.0033,C0
4,f00,f11,f20,f30,f40,f50,f60,f71,f80,0.1692,0.2393,2.5039,0.8708,1.6206,0.6319,0.6632,0.6857,0.6148,C0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,f01,f10,f20,f30,f41,f50,f61,f71,f80,0.3347,0.7849,0.0796,0.7342,0.5169,0.7413,0.4696,0.6958,0.6273,C4
3996,f00,f10,f20,f30,f41,f50,f61,f70,f80,0.2989,0.6772,0.0325,2.4367,0.2669,1.1615,0.4936,0.3491,0.8280,C4
3997,f00,f10,f20,f31,f41,f50,f61,f71,f80,0.5245,0.6542,1.0248,1.4574,0.1816,0.5517,0.1486,0.3595,1.6582,C4
3998,f00,f10,f20,f30,f41,f50,f61,f70,f80,0.7995,0.2713,0.4453,1.1236,1.0424,0.4081,1.0728,0.5401,1.0678,C4


In [50]:
# 打亂數據
df = shuffle(df, random_state=42)
df


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,class
555,f00,f11,f21,f30,f41,f50,f61,f70,f80,0.2180,1.7515,1.3247,0.7045,0.1602,0.2377,0.0604,0.4097,0.1413,C0
3491,f00,f10,f20,f30,f40,f51,f60,f70,f80,0.2405,0.2647,1.2717,0.0171,0.0327,0.9587,0.7820,0.1257,1.0321,C4
527,f01,f10,f21,f30,f41,f50,f61,f71,f80,0.8994,0.7148,0.0695,0.8427,1.8601,1.5520,1.0055,0.4666,0.8031,C0
3925,f00,f10,f20,f30,f41,f50,f61,f70,f80,1.2039,0.2248,0.6012,0.3104,1.5987,0.7372,0.4647,0.3928,2.0255,C4
2989,f00,f10,f21,f31,f40,f50,f61,f71,f80,1.0685,1.7426,0.0985,1.4925,1.9991,0.7789,0.8078,0.5173,9.2294,C3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,f01,f11,f20,f30,f40,f50,f61,f71,f80,1.9241,2.4659,1.2694,0.9267,0.0465,0.5840,0.7568,0.1007,0.4976,C1
1294,f00,f11,f20,f30,f40,f50,f61,f70,f81,0.5942,2.3849,0.0990,0.5941,1.9762,0.6852,1.0806,1.0973,0.9019,C1
860,f00,f10,f21,f30,f40,f50,f61,f71,f80,1.7290,0.5702,0.4253,1.9224,1.6729,0.3761,0.1195,0.2926,2.3368,C1
3507,f00,f10,f20,f30,f41,f50,f61,f71,f80,0.4379,0.8253,0.6256,2.1693,0.7944,0.6540,0.1803,0.8967,1.0243,C4


In [51]:
# 檢查是否有缺失值
print(df.isnull().sum())

# 處理缺失值（這裡我們選擇用均值填補缺失值）
df.fillna(df.mean(), inplace=True)

f0       0
f1       0
f2       0
f3       0
f4       0
f5       0
f6       0
f7       0
f8       0
f9       0
f10      0
f11      0
f12      0
f13      0
f14      0
f15      0
f16      0
f17      0
class    0
dtype: int64


C:\Users\nonohuang\AppData\Local\Temp\ipykernel_18780\3699161564.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.mean(), inplace=True)


In [52]:
# 分割特徵和標籤
X = df.drop('class', axis=1)
y = df['class']


In [53]:
# 使用LabelEncoder對類別型特徵進行編碼
label_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = le

In [54]:
# 標籤編碼
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 分割資料為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [55]:
# 建立SVC分類模型
model = SVC()

# 訓練模型
model.fit(X_train, y_train)

# 預測
y_pred = model.predict(X_test)

# 評估模型
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# 進行交叉驗證
scores = cross_val_score(model, X, y_encoded, cv=5)
print(f'Cross-validation scores: {scores}')
print(f'Average cross-validation score: {scores.mean()}')

Accuracy: 0.815
Cross-validation scores: [0.825   0.79375 0.8125  0.82375 0.7825 ]
Average cross-validation score: 0.8074999999999999


In [56]:
# 讀取測試數據
filepath_test = r"C:\Users\nonohuang\OneDrive\桌面\kaggle\kaggle\task2\introml_2024_task2_test_NO_answers_shuffled.csv"
df_test = pd.read_csv(filepath_test)

# 刪除 id 特徵
X_test = df_test.drop(columns=['id'])

# 使用與訓練數據相同的編碼器對測試數據進行編碼
for column in X_test.select_dtypes(include=['object']).columns:
    le = label_encoders[column]
    X_test[column] = le.transform(X_test[column])

# 預測
test_predictions = model.predict(X_test)

# 將預測結果轉換回原始標籤
test_predictions_labels = label_encoder.inverse_transform(test_predictions)

# 將預測結果保存到csv文件
submission = pd.DataFrame({'id': df_test['id'], 'class': test_predictions_labels})
submission.to_csv(r'C:\Users\nonohuang\OneDrive\桌面\kaggle\kaggle\task2\submission\submission.csv', index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv
